In [28]:
import pandas as pd
from openai import OpenAI
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



In [7]:
# Load the Excel file
file_path = 'Sample.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,QuestionId,AcceptedAnswerId,QuestionBody,QuestionTitle,QuestionTags,QuestionPostTypeId,QuestionOwnerUserId,QuestionScore,AnswerId,AnswerBody,AnswerTitle,AnswerTags,AnswerPostTypeId,AnswerOwnerUserId,AnswerScore,UserId,Reputation,UpVotes,DownVotes,Views
0,32921922,32924584,<p>What are the names of the trigonometric fun...,How to use trigonometry in Scheme?,<scheme><trigonometry>,1,5223757.0,0,32924584,<p>The Scheme standards define six trigonometr...,NaN,NaN,2,13,4,13,220982,5212,210,46020
1,32800446,32802138,<p>I've checked out the docs for this on docs....,Javascript-esq random floating point number 0-...,<random><floating-point><racket>,1,2124732.0,0,32802138,"<p>Yes, just use <code>(random)</code>. Exampl...",NaN,NaN,2,13,3,13,220982,5212,210,46020
2,32751289,32751306,"<p>I'm new to java, I'm trying to select shape...",I have an error with my if statement,<java><swing>,1,3785208.0,-1,32751306,<p>(Style nit: your <code>design</code> class ...,NaN,NaN,2,13,0,13,220982,5212,210,46020
3,32724801,32724854,<p>How is even possible to .NET's P/invoke cal...,How p/invoke call function using only but C++ ...,<c#><c++><c++-cli><mingw><pinvoke>,1,800123.0,-1,32724854,<p>You <em>can</em> use <code>GetProcAddress</...,NaN,NaN,2,13,1,13,220982,5212,210,46020
4,32717174,32717341,<p>I am encountering a problem trying to repla...,Two equal strings representing whitespaces not...,<javascript><string><numbers><locale>,1,2550819.0,4,32717341,"<p>In my testing, the separator character is a...",NaN,NaN,2,13,1,13,220982,5212,210,46020


In [8]:

# Function to remove HTML tags from the text
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Apply the cleaning function to the QuestionBody
data['CleanedQuestionBody'] = data['QuestionBody'].apply(clean_html)

# Display the cleaned question bodies and the associated user IDs
data[['CleanedQuestionBody', 'AnswerOwnerUserId']].head()


,CleanedQuestionBody,AnswerOwnerUserId
0,What are the names of the trigonometric functi...,13
1,I've checked out the docs for this on docs.rac...,13
2,"I'm new to java, I'm trying to select shape us...",13
3,How is even possible to .NET's P/invoke call a...,13
4,I am encountering a problem trying to replace ...,13


In [21]:
client = OpenAI(api_key='')

# Function to generate embeddings
def generate_embeddings(texts, model="text-embedding-3-small"):
    response = client.embeddings.create(
        input=texts,
        model=model
    ).data[0].embedding
    return response



In [22]:
# Assuming 'data' is your DataFrame and it includes a 'CleanedQuestionBody' column
questions = data['CleanedQuestionBody'].tolist()
embeddings = generate_embeddings(questions)

print(embeddings)



[-0.055548299103975296, 0.029936717823147774, 0.0263112373650074, -0.01653728075325489, 0.04448104277253151, 0.024318281561136246, -0.018191007897257805, -0.02510274387896061, 0.015413593500852585, -0.008740376681089401, -0.011173265054821968, -0.0384809784591198, 0.0020989624317735434, -0.0006108722882345319, 0.030848387628793716, 0.00020737855811603367, -0.05656597763299942, 0.03975307568907738, -0.015222778543829918, 0.01573161780834198, -0.008014220744371414, -0.005448821932077408, 0.00948773417621851, 0.02845260314643383, 0.029343070462346077, 0.04609236866235733, -0.009779256768524647, -0.002297727856785059, 0.01007607951760292, -0.0338166169822216, 0.027265310287475586, -0.02065039798617363, -0.024996735155582428, -0.03404983505606651, -0.0028569211717694998, 0.027964964509010315, -0.03882020339369774, 0.019600916653871536, -0.007791602984070778, 0.023406611755490303, -0.034855496138334274, -0.020364176481962204, -0.05622675269842148, -0.034855496138334274, -0.030742378905415535

In [26]:
data['ada_embedding'] = data.CleanedQuestionBody.apply(lambda x: generate_embeddings(x, model='text-embedding-3-small'))


OSError: Cannot save file into a non-existent directory: 'output'

In [27]:
data.to_csv('output/embedded_1k_reviews.csv', index=False)

In [29]:

# Function to find similar questions
def find_similar_questions(query_embedding, embeddings, top_k=5):
    similarities = cosine_similarity([query_embedding], embeddings)
    similar_indices = np.argsort(-similarities[0])[:top_k]
    return similar_indices


In [46]:
# Example usage
new_question = "How do I center a div"
new_embedding = generate_embeddings([new_question])
similar_question_indices = find_similar_questions(new_embedding, np.array(data['ada_embedding'].tolist()))
similar_user_ids = data.iloc[similar_question_indices][['CleanedQuestionBody', 'AnswerOwnerUserId']]

In [48]:
similar_user_ids

,CleanedQuestionBody,AnswerOwnerUserId
115,How do I\n\nvertically center an image (crop t...,371
508,What I am trying to achieve is a container wit...,1946
373,"I have two divs with child elements, I wish to...",1324
344,Below you can see my code. It seems that the f...,1324
384,I am trying to get the counterpart to selected...,1324


In [47]:
similar_user_ids.iloc[1,0]

'What I am trying to achieve is a container with three inline divs that expand on the whole container when the mouse is hovered on them. One of the divs is floated to the right, one to the left, and one is centred. Here is a jsfiddle page so you can understand better what I am trying. \nhttps://jsfiddle.net/hn59ooL5/\n<div id="container">\n        <div id="container-left">\n            <div id="info-img-container" onmouseover="hoverOverLeft();" onmouseout="unHoverLeft();">\n                <img id="info-img" src="images/info.png" alt="info" name="info"/>\n            </div>\n            <div id="info-text-container">\n                Hover your mouse over this card to find info on how to use this website\n            </div>\n        </div>\n\n        <div id="container-right">\n            container right\n        </div>\n\n        <div id="container-center">\n\n        </div>\n\n\n    </div>\n\nand the css:\n#container{\nmargin: auto;\nwidth: 85%;\nheight:55%;\ndisplay: block;\npositi